In [147]:
import os
import glob
import hdf5_getters
import time

import numpy as np
from scipy import stats

In [5]:
h5 = hdf5_getters.open_h5_file_read('./MillionSongSubset/AdditionalFiles/subset_msd_summary_file.h5')

In [76]:
h5.root

/ (RootGroup) 'H5 Song File'
  children := ['musicbrainz' (Group), 'analysis' (Group), 'metadata' (Group)]

In [79]:
h5.root.musicbrainz

/musicbrainz (Group) 'data about the song coming from MusicBrainz'
  children := ['songs' (Table)]

In [80]:
h5.root.musicbrainz.songs

/musicbrainz/songs (Table(10000,), shuffle, zlib(1)) 'table of data coming from MusicBrainz'
  description := {
  "idx_artist_mbtags": Int32Col(shape=(), dflt=0, pos=0),
  "year": Int32Col(shape=(), dflt=0, pos=1)}
  byteorder := 'little'
  chunkshape := (1024,)

In [77]:
h5.root.metadata

/metadata (Group) 'metadata about the song'
  children := ['songs' (Table)]

In [73]:
h5.root.metadata.songs

/metadata/songs (Table(10000,), shuffle, zlib(1)) 'table of metadata for one song'
  description := {
  "analyzer_version": StringCol(itemsize=32, shape=(), dflt='', pos=0),
  "artist_7digitalid": Int32Col(shape=(), dflt=0, pos=1),
  "artist_familiarity": Float64Col(shape=(), dflt=0.0, pos=2),
  "artist_hotttnesss": Float64Col(shape=(), dflt=0.0, pos=3),
  "artist_id": StringCol(itemsize=32, shape=(), dflt='', pos=4),
  "artist_latitude": Float64Col(shape=(), dflt=0.0, pos=5),
  "artist_location": StringCol(itemsize=1024, shape=(), dflt='', pos=6),
  "artist_longitude": Float64Col(shape=(), dflt=0.0, pos=7),
  "artist_mbid": StringCol(itemsize=40, shape=(), dflt='', pos=8),
  "artist_name": StringCol(itemsize=1024, shape=(), dflt='', pos=9),
  "artist_playmeid": Int32Col(shape=(), dflt=0, pos=10),
  "genre": StringCol(itemsize=1024, shape=(), dflt='', pos=11),
  "idx_artist_terms": Int32Col(shape=(), dflt=0, pos=12),
  "idx_similar_artists": Int32Col(shape=(), dflt=0, pos=13),
  "relea

In [78]:
h5.root.analysis

/analysis (Group) 'Echo Nest analysis of the song'
  children := ['songs' (Table)]

In [74]:
h5.root.analysis.songs

/analysis/songs (Table(10000,), shuffle, zlib(1)) 'table of Echo Nest analysis for one song'
  description := {
  "analysis_sample_rate": Int32Col(shape=(), dflt=0, pos=0),
  "audio_md5": StringCol(itemsize=32, shape=(), dflt='', pos=1),
  "danceability": Float64Col(shape=(), dflt=0.0, pos=2),
  "duration": Float64Col(shape=(), dflt=0.0, pos=3),
  "end_of_fade_in": Float64Col(shape=(), dflt=0.0, pos=4),
  "energy": Float64Col(shape=(), dflt=0.0, pos=5),
  "idx_bars_confidence": Int32Col(shape=(), dflt=0, pos=6),
  "idx_bars_start": Int32Col(shape=(), dflt=0, pos=7),
  "idx_beats_confidence": Int32Col(shape=(), dflt=0, pos=8),
  "idx_beats_start": Int32Col(shape=(), dflt=0, pos=9),
  "idx_sections_confidence": Int32Col(shape=(), dflt=0, pos=10),
  "idx_sections_start": Int32Col(shape=(), dflt=0, pos=11),
  "idx_segments_confidence": Int32Col(shape=(), dflt=0, pos=12),
  "idx_segments_loudness_max": Int32Col(shape=(), dflt=0, pos=13),
  "idx_segments_loudness_max_time": Int32Col(shape=()

In [120]:
tids = [(row['track_id'], row['tempo']) for row in h5.root.analysis.songs.where('tempo>245')]
#'(tempo>245) & (tempo<250)')]
tids

[('TRAFJKG128F9332800', 246.5),
 ('TRAZFZS128F4272A31', 246.593),
 ('TRAGYJM128EF3466D2', 253.357),
 ('TRBHZRN128F92EFF91', 258.677),
 ('TRBFXLH12903CBAFAD', 248.079),
 ('TRBGYHC12903D0626A', 262.828)]

In [105]:
tids_titles = []

def get_items(t_id, titles, metas, sh5):
    title = hdf5_getters.get_title(sh5)
    artist = hdf5_getters.get_artist_name(sh5)
    artist_id = hdf5_getters.get_artist_id(sh5)
    song_id = hdf5_getters.get_song_id(sh5)
    titles.append((t_id, title, artist_id, artist, song_id))
    song_hottt = hdf5_getters.get_song_hotttnesss(sh5)
    artist_hottt = hdf5_getters.get_artist_hotttnesss(sh5)
    artist_fam = hdf5_getters.get_artist_familiarity(sh5)
    metas.append((t_id, song_hottt, artist_hottt, artist_fam))
                    
def get_matching_titles(track_ids=None, basedir='./MillionSongSubset/data/',ext='.h5', verbose=0):
    t1 = time.time()
    titles = []
    metas = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files:
            if len(titles) == len(track_ids):
                break
            sh5 = hdf5_getters.open_h5_file_read(f)
            if track_ids:
                for t_id in track_ids:
                    if hdf5_getters.get_track_id(sh5) == t_id:
                        get_items(t_id, titles, metas, sh5)
                        break
            else:
                t_id = hdf5_getters.get_track_id(sh5)
                get_items(t_id, titles, metas, sh5)
            sh5.close()
    t2 = time.time()
    if verbose == 1:
        print "total time: ", (t2-t1)
    return titles, metas

tids_only = [t[0] for t in tids]
titles, metas = get_matching_titles(track_ids=tids_only, verbose=1)
print titles
print metas

total time:  66.7843520641
([('TRBGYHC12903D0626A', 'Endless Light', 'AR0JEOI1187B98B918', 'Erik Berglund', 'SODWYEY12AC468C30F'), ('TRBHZRN128F92EFF91', 'Kapitel 4', 'ARIEIJW1187B98F6DD', 'Oliver Kalkofe', 'SOQFMJN12AB0181D3D'), ('TRBFXLH12903CBAFAD', "Two of a Kind_ Workin' on a Full House", 'ARBMHBC11F4C8403A2', 'Hushabye Baby', 'SOOMCCM12AB01896EA'), ('TRAZFZS128F4272A31', 'Severe Severing', 'ARSL3N21187B98DFC5', 'Klaus Badelt', 'SOABYBF12A8C138A09'), ('TRAGYJM128EF3466D2', 'Been There All The Time', 'AR9YWMS1187FB43A34', 'Dinosaur Jr.', 'SOZRIFJ12A67ADA4BE'), ('TRAFJKG128F9332800', 'Ego is the drug/3am', 'ARCRSUM1187FB52AB0', 'The Frequency', 'SONNXDI12AB0186816')], [('TRBGYHC12903D0626A', 0.0, 0.33619115568493391, 0.37703408512839853), ('TRBHZRN128F92EFF91', 0.21508031850922793, 0.40619874783410698, 0.45248343829809812), ('TRBFXLH12903CBAFAD', nan, 0.36551400326265299, 0.35342955022197026), ('TRAZFZS128F4272A31', 0.32773668317784083, 0.46934037425924607, 0.68800334883697312), ('T

In [126]:
seg_loud_max = [(row['track_id'], row['idx_sections_start']) for row in h5.root.analysis.songs.where('(tempo>238) & (tempo<240)')]
print seg_loud_max

[('TRAWVUW128F932F914', 0), ('TRBFFKN128F422F3CC', 0)]


In [171]:
def get_pitch_features(sh5):
    all_seg_pitches = hdf5_getters.get_segments_pitches(sh5)
    pitch_means = np.mean(seg_pitches, axis=0).tolist()
    pitch_medians = np.median(seg_pitches, axis=0).tolist()
    pitch_modes = stats.mode(seg_pitches)[0].tolist()[0]
    return pitch_means, pitch_medians, pitch_modes

def get_loudness_features(sh5):
    loud_maxs = hdf5_getters.get_segments_loudness_max(h5)
    loud_mean = np.mean(loud_maxs)
    loud_median = np.median(loud_maxs)
    loud_mode = stats.mode(loud_maxs)[0].tolist()[0]
    return loud_mean, loud_median, loud_mode

def get_segment_features(track_ids=None, basedir='./MillionSongSubset/data/',ext='.h5', verbose=0):
    t1 = time.time()
    titles = []
    metas = []
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files:
            if len(titles) == len(track_ids):
                break
            sh5 = hdf5_getters.open_h5_file_read(f)
            if track_ids:
                for t_id in track_ids:
                    if hdf5_getters.get_track_id(sh5) == t_id:
                        pitch_means, pitch_medians, pitch_modes = get_pitch_features(sh5)
                        return get_loudness_features(sh5)
                        break
            else:
                t_id = hdf5_getters.get_track_id(sh5)
                pitch_means, pitch_medians, pitch_modes = get_pitch_features(sh5)
            sh5.close()
    t2 = time.time()
    if verbose == 1:
        print "total time: ", (t2-t1)
    return 

tids_only0 = [t[0] for t in tids][2:3]
print tids_only0
print get_segment_features(track_ids=tids_only0, verbose=1)

['TRAGYJM128EF3466D2']


NoSuchNodeError: group ``/analysis`` does not have a child named ``segments_loudness_max``

In [160]:
a = np.array([[1, 2], [3, 4], [5, 6]])
print np.mean(a, axis=0)
print np.mean(a, axis=1)

[ 3.  4.]
[ 1.5  3.5  5.5]


In [166]:
print seg_pitches.shape
print type(seg_pitches)
seg_pitch_mean = np.mean(seg_pitches, axis=0)
print seg_pitch_mean.shape, "\n", tuple(seg_pitch_mean)
print seg_pitches
print
print tuple(np.median(seg_pitches, axis=0))
print
print tuple(stats.mode(seg_pitches)[0].tolist()[0])


(530, 12)
<type 'numpy.ndarray'>
(12,) 
(0.46639056603773582, 0.55831886792452867, 0.59365471698113281, 0.3651773584905661, 0.42963396226415096, 0.33559433962264151, 0.38138679245283025, 0.33712452830188638, 0.44591320754717012, 0.68786603773584898, 0.42415094339622633, 0.48663962264150951)
[[ 0.833  0.333  0.224 ...,  0.614  1.     0.189]
 [ 0.299  0.195  0.228 ...,  0.73   0.423  0.217]
 [ 0.333  0.387  0.364 ...,  0.971  0.488  0.317]
 ..., 
 [ 0.112  0.167  1.    ...,  0.483  0.104  0.182]
 [ 0.536  1.     0.764 ...,  0.52   0.189  0.16 ]
 [ 0.07   0.077  0.547 ...,  1.     0.332  0.239]]

(0.42949999999999999, 0.502, 0.5555000000000001, 0.35149999999999998, 0.38700000000000001, 0.314, 0.32750000000000001, 0.29449999999999998, 0.40500000000000003, 0.74849999999999994, 0.39700000000000002, 0.443)

(1.0, 1.0, 1.0, 0.034, 1.0, 0.318, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0)
